In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.decomposition import PCA
import numpy as np
import pickle
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance

In [3]:
#Функция для перевода строки новых ненормализованных данных в строку нормализованных
# features здесь это только числовые параметры, в том же порядке в котором они записаны в scaler (а значит в массив "numeric_features")
def new_scaler(features, scale, mean):
    for i in range(len(features)):
        features[i] = (features[i] - mean[i])/ scale[i]
    return features

#Функция для перевода строки нормализованных данных в первоночальный вид
def scaled_to_normal(features, scale, mean):
    for i in range(len(features)):
        features[i] = features[i] * scale[i] + mean[i]
    return features

In [17]:
#Чтение данных

data = pd.read_csv('train_short.csv')
# Показать названия всех столбцов
print("\nНазвания столбцов:")
print(data.columns.tolist())

# Показать первые несколько строк
print("\nПервые 2 строк данных:")
print(data.head(2))


Названия столбцов:
['id', 'Gender', 'Age', 'Driving_License', 'Region_Code', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage', 'Response']

Первые 2 строк данных:
   id Gender  Age  Driving_License  Region_Code  Previously_Insured  \
0   0   Male   21                1         35.0                   0   
1   1   Male   43                1         28.0                   0   

  Vehicle_Age Vehicle_Damage  Annual_Premium  Policy_Sales_Channel  Vintage  \
0    1-2 Year            Yes         65101.0                 124.0      187   
1   > 2 Years            Yes         58911.0                  26.0      288   

   Response  
0         0  
1         1  


In [19]:
# Удаляем строки с пропущенными значениями (их нет)
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000001 entries, 0 to 5000000
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   Gender                object 
 2   Age                   int64  
 3   Driving_License       int64  
 4   Region_Code           float64
 5   Previously_Insured    int64  
 6   Vehicle_Age           object 
 7   Vehicle_Damage        object 
 8   Annual_Premium        float64
 9   Policy_Sales_Channel  float64
 10  Vintage               int64  
 11  Response              int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 457.8+ MB


In [21]:
#Оставляем только нужные столбцы
#columns_to_keep = ['Response', 'Gender', 'Age', 'Driving_License', 'Region_Code', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage']
columns_to_keep = ['Response', 'Gender', 'Age', 'Driving_License', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Annual_Premium']
#columns_to_keep = ['Response', 'Gender', 'Age', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Policy_Sales_Channel']
data = data[columns_to_keep]

In [31]:
#testD = pd.read_csv('test_short.csv')
#testD = testD[columns_to_keep]
#testD.head()

In [23]:
# One-hot кодирование категориальных признаков
categorical_features = ['Gender', 'Vehicle_Age', 'Vehicle_Damage', 'Driving_License', 'Previously_Insured']
#categorical_features = ['Gender', 'Vehicle_Age', 'Vehicle_Damage', 'Previously_Insured']

data = pd.get_dummies(data, columns=categorical_features, drop_first=True)

length = len(data)

In [36]:
#testD = pd.get_dummies(testD, columns=categorical_features, drop_first=True)

In [25]:
# Нормализация числовых признаков
#numeric_features = ["Age", 'Region_Code', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage']
numeric_features = ["Age", 'Annual_Premium']
#numeric_features = ["Age", 'Policy_Sales_Channel']
scaler = StandardScaler()
data[numeric_features] = scaler.fit_transform(data[numeric_features])

In [38]:
# scaler.scale_

In [39]:
# scaler.mean_

In [40]:
#testD[numeric_features] = scaler.transform(testD[numeric_features])

In [47]:
#Функция для удаления выбросов. Проверяем только численные значения
'''def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    #Можно тут поиграться с коэффициентом на который умножается
    lower_bound = Q1 - 1.6 * IQR
    upper_bound = Q3 + 1.6 * IQR
    
    return df[(df[column] >= lower_bound) & 
             (df[column] <= upper_bound)]

data_clean = data
#Удаление выбросов
for col in data_clean.select_dtypes(include=['float64']).columns:
    data_clean = remove_outliers_iqr(data_clean, col)

print("Строчек удалено: ", len(data) - len(data_clean))
print(len(data_clean))
print(len(data))'''

'def remove_outliers_iqr(df, column):\n    Q1 = df[column].quantile(0.25)\n    Q3 = df[column].quantile(0.75)\n    IQR = Q3 - Q1\n    \n    #Можно тут поиграться с коэффициентом на который умножается\n    lower_bound = Q1 - 1.6 * IQR\n    upper_bound = Q3 + 1.6 * IQR\n    \n    return df[(df[column] >= lower_bound) & \n             (df[column] <= upper_bound)]\n\ndata_clean = data\n#Удаление выбросов\nfor col in data_clean.select_dtypes(include=[\'float64\']).columns:\n    data_clean = remove_outliers_iqr(data_clean, col)\n\nprint("Строчек удалено: ", len(data) - len(data_clean))\nprint(len(data_clean))\nprint(len(data))'

In [49]:
'''data = data_clean'''

'data = data_clean'

In [27]:
#Данные несбалансированные
data.Response.value_counts()

Response
0    4384720
1     615281
Name: count, dtype: int64

In [29]:
data.head()

,Response,Age,Annual_Premium,Gender_Male,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years,Vehicle_Damage_Yes,Driving_License_1,Previously_Insured_1
0,0,-1.159421,2.101969,True,False,False,True,True,False
1,1,0.307563,1.726275,True,False,True,True,True,False
2,0,-0.892697,0.459721,False,True,False,False,True,True
3,0,-0.225886,-1.689622,False,False,False,True,True,False
4,0,-0.159205,0.089975,False,False,False,False,True,True


In [35]:
# Разделение на признаки (X) и целевую переменную (y)
X = data.drop('Response', axis=1)
y = data['Response']

# Разделение на обучающую и тестовую выборки. Только для проверки PCA, потом надо заменить на кросс-валидацию
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y)

In [45]:
#X2 = testD.drop('Response', axis=1)
#y2 = testD['Response']
#X2.to_pickle('X_train_phy.pkl')
#y2.to_pickle('y_train.pkl')

In [ ]:
n = 6
# Создаем объект PCA с n компонентами
pca = PCA(n_components=n)

# Применяем PCA к данным
pca_data = pca.fit_transform(X_train)

In [20]:
# Просто смотрю как в целом обучать модель используя pca. Потом уберем
# Не советую расскоментировать тк на 5миллионах данных считается очень долго, я на меньшем количестве проверяла и оно работает

"""
X_train = pca.fit_transform(X_train)

knn_param_grid = {'n_neighbors': [3]}
knn_model = GridSearchCV(KNeighborsClassifier(), knn_param_grid, cv=5)
knn_model.fit(X_train, y_train)

X_test = pca.fit_transform(X_test)
y_pred = knn_model.predict(X_test)
# Отчет классификации
print(classification_report(y_test, y_pred))
"""

"\nX_train = pca.fit_transform(X_train)\n\nknn_param_grid = {'n_neighbors': [3]}\nknn_model = GridSearchCV(KNeighborsClassifier(), knn_param_grid, cv=5)\nknn_model.fit(X_train, y_train)\n\nX_test = pca.fit_transform(X_test)\ny_pred = knn_model.predict(X_test)\n# Отчет классификации\nprint(classification_report(y_test, y_pred))\n"

In [51]:
import pickle

# Сохранение преобразования в файл
'''filename = "scaler.pkl"
with open(filename, 'wb') as file:
    pickle.dump(scaler, file)
print(f"scaler сохранена в файл: {filename}")

filename = "pca.pkl"
with open(filename, 'wb') as file:
    pickle.dump(pca, file)
print(f"PCA сохранена в файл: {filename}")'''

'filename = "scaler.pkl"\nwith open(filename, \'wb\') as file:\n    pickle.dump(scaler, file)\nprint(f"scaler сохранена в файл: {filename}")\n\nfilename = "pca.pkl"\nwith open(filename, \'wb\') as file:\n    pickle.dump(pca, file)\nprint(f"PCA сохранена в файл: {filename}")'

In [65]:
#X.head()
#X.to_pickle('X_train_phy.pkl')
#y.to_pickle('y_train.pkl')

In [63]:
X_train = pca.transform(X)
y_train = y

In [69]:
#with open('X_train_pca.pkl', 'wb') as file:
#    pickle.dump(X_train, file)

In [ ]:
# 1. Decision Tree
t_start = time.time()
tree_param_grid = {'max_depth': [10, 15], 'min_samples_split': [20, 50]}
tree_model = GridSearchCV(DecisionTreeClassifier(), tree_param_grid, scoring = 'f1', cv=5)
tree_model.fit(X_train, y_train)
t_finish = time.time()
dt = t_finish - t_start
dt2 = dt / 60.
print("Decision Tree Best Params:", tree_model.best_params_)
print("Time: ",  dt, " sec or ", dt2, "min")

In [ ]:
y_lab = tree_model.predict(X)

In [53]:
print(tree_model.score(X, y))

0.01555679353453881


In [41]:
print(f1_score(y, y_lab))

0.021228627169638876


In [55]:
print(confusion_matrix(y, y_lab))

[[4381967    2753]
 [ 610436    4845]]


In [ ]:
# 2. Random Forest
#rf_param_grid = {'n_estimators': [10, 50, 100, 200], 'max_depth': [5, 10, 15], 'min_samples_split': [3, 5, 10]}
#rf_model = GridSearchCV(RandomForestClassifier(random_state=42), rf_param_grid, cv=5)
#rf_model.fit(X_train, y_train)
#print("Random Forest Best Params:", rf_model.best_params_)

In [ ]:
# other variants -- 
# Naive Bayes: https://scikit-learn.org/stable/modules/naive_bayes.html
# other ensembles : https://scikit-learn.org/stable/modules/ensemble.html
# Мне кажется, лучше не пробовать boosting

In [45]:
# Сохранение модели в файл
model_filename = "tree1_model.pkl"
with open(model_filename, 'wb') as file:
    pickle.dump(tree_model, file)
print(f"Модель сохранена в файл: {model_filename}")

Модель сохранена в файл: tree1_model.pkl


GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False


scoring : https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

‘balanced_accuracy’
‘f1’ 
‘precision’ -- Нужно смотреть, но не знаю, как пользоваться. Он сказал, важнее 'recall', но потом значит, что нам нужно свой программировать. 
(‘roc_auc’ тоже существует.)

n_jobs, pre_dispatch наверно полезно